# Factory Demo

In [42]:
from patterns.Factory_InstrumentTypes import InstrumentFactory

instruments = InstrumentFactory.load_from_csv('inputs/instruments.csv')

print(instruments[0].instrument_type)
print(instruments[0].symbol)

custom_data = {
    'symbol': 'TSLA',
    'type': 'stock',
    'price': '250.00',
    'sector': 'Automotive',
    'issuer': 'Tesla Inc.'
}

custom_stock = InstrumentFactory.create_instrument(custom_data)

print("--")

print(custom_stock.symbol)


stock
AAPL
--
TSLA


# Singleton

In [43]:
from patterns.Singleton_ConfigAccess import Config

config = Config()

# Access
# print(config.config)

print(config.config.get('log_level'))
# print(config.config.get('data_path'))
# print(config.config.get('report_path'))

# Check:
s1 = Config()
s2 = Config()

print(s1 is s2)  # Output: True (both variables point to the same instance)

INFO
True


# Builder

In [44]:
from patterns.Builder_PortfolioBuilder import PortfolioBuilder, Director

director = Director()

# from portfolio_structure.json
portfolio = director.build_portfolio()


print(portfolio.name)
print(portfolio.owner)
print(portfolio.positions)
# subport access
print("--")
print(portfolio.subportfolios)
print(portfolio.subportfolios[0]['subportfolio'].positions)
# print(portfolio.subportfolios[0].positions)



Main Portfolio
sdonadio
[{'symbol': 'AAPL', 'quantity': 100, 'price': 172.35}, {'symbol': 'MSFT', 'quantity': 50, 'price': 328.1}]
--
[{'name': 'Index Holdings', 'subportfolio': <patterns.Builder_PortfolioBuilder.Portfolio object at 0x1174e9810>}]
[{'symbol': 'SPY', 'quantity': 20, 'price': 430.5}]


# Decorator

# Adapter

In [50]:
import importlib
# from Adapter_DataLoader import YahooFinanceAdapter, BloombergXMLAdapter
import Adapter_DataLoader

importlib.reload(Adapter_DataLoader)

yahoo_adapter = Adapter_DataLoader.YahooFinanceAdapter()

print(yahoo_adapter.get_data('AAPL').symbol)
print(yahoo_adapter.get_data('AAPL').price)
print(yahoo_adapter.get_data('AAPL').timestamp)

bloomberg_adapter = Adapter_DataLoader.BloombergXMLAdapter()

print(bloomberg_adapter.get_data('MSFT').symbol)
print(bloomberg_adapter.get_data('MSFT').price)
print(bloomberg_adapter.get_data('MSFT').timestamp)




AAPL
172.35
2025-10-01T09:30:00Z
MSFT
328.1
2025-10-01T09:30:00Z


# Composite

In [54]:
from patterns.Composite_PortModel import build_portfolio_from_json

portfolio = build_portfolio_from_json('inputs/portfolio_structure.json')

print(portfolio.name)
print(portfolio.get_value())

for pos in portfolio.get_positions():
    print(pos)


Main Portfolio
42250.0
{'symbol': 'AAPL', 'quantity': 100, 'price': 172.35}
{'symbol': 'MSFT', 'quantity': 50, 'price': 328.1}
{'symbol': 'SPY', 'quantity': 20, 'price': 430.5}
